In [1]:
pip install --upgrade openai python-dotenv pandas tqdm tenacity

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import json
import pandas as pd
from pathlib import Path
from datetime import datetime

from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from openai import OpenAI


In [4]:
# Load the .env file
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    print("python-dotenv not installed or .env missing")

# Fetch the key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("OPENAI_API_KEY loaded:", "YES" if OPENAI_API_KEY else "NO")


OPENAI_API_KEY loaded: YES


In [5]:
if not OPENAI_API_KEY:
    raise ValueError("❌ API Key not found. Make sure it's in your .env file as OPENAI_API_KEY.")

client = OpenAI(api_key=OPENAI_API_KEY)

# Tiny check call
try:
    resp = client.responses.create(
        model="gpt-4.1-mini",
        input="Hello! Just checking if the API key works."
    )
    print("API Key check successful ✔️")
except Exception as e:
    print("❌ Key failed:", e)


API Key check successful ✔️


In [6]:
# List all available OpenAI models with categories

from openai import OpenAI
import pandas as pd

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

models = client.models.list()

model_data = []

for m in models.data:
    model_id = m.id

    # Categorize based on name patterns
    if "gpt" in model_id.lower() or "o1" in model_id.lower():
        category = "Chat / Reasoning Models"
    elif "embed" in model_id.lower():
        category = "Embedding Models"
    elif "tts" in model_id.lower() or "audio" in model_id.lower():
        category = "Audio / Speech Models"
    elif "vision" in model_id.lower() or "clip" in model_id.lower():
        category = "Vision Models"
    else:
        category = "Other"

    model_data.append({
        "model_id": model_id,
        "category": category
    })

df_models = pd.DataFrame(model_data).sort_values("model_id")
df_models.reset_index(drop=True, inplace=True)

df_models


,model_id,category
0,babbage-002,Other
1,chatgpt-4o-latest,Chat / Reasoning Models
2,codex-mini-latest,Other
3,dall-e-2,Other
4,dall-e-3,Other
...,...,...
95,tts-1,Audio / Speech Models
96,tts-1-1106,Audio / Speech Models
97,tts-1-hd,Audio / Speech Models
98,tts-1-hd-1106,Audio / Speech Models


In [7]:
# Check if o3-mini is available in YOUR account and set MODEL_ID accordingly

want = "o3-mini"
has_o3_mini = (df_models["model_id"].str.contains(r"\bo3[-_]mini\b", case=False, na=False)).any()

if has_o3_mini:
    o3_row = df_models[df_models["model_id"].str.contains(r"\bo3[-_]mini\b", case=False, na=False)].sort_values("model_id").head(1)
    MODEL_ID = o3_row.iloc[0]["model_id"]
    print(f"✅ Found {MODEL_ID} in your account. Using this model.")
else:
    # sensible fallback if you don't have access
    MODEL_ID = "gpt-4.1-mini"
    print("⚠️ o3-mini not found on this account. Falling back to:", MODEL_ID)

MODEL_ID


✅ Found o3-mini in your account. Using this model.


'o3-mini'

In [8]:
MODEL_ID = "o3-mini"
print("Model selected:", MODEL_ID)


Model selected: o3-mini


In [9]:
from openai import OpenAI
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "❌ Missing OPENAI_API_KEY"

client = OpenAI(api_key=OPENAI_API_KEY)

try:
    resp = client.responses.create(
        model=MODEL_ID,
        input="Quick hello — please reply with 'OK' if you can hear me."
    )
    print("o3-mini check OK ✅")
    print("Sample output:", resp.output_text[:120].replace("\n"," "))
except Exception as e:
    print("❌ Model check failed:", e)


o3-mini check OK ✅
Sample output: OK


In [10]:
def build_user_prompt(title: str, body: str) -> str:
    return f"""
You are a trauma-informed, non-judgmental peer-support assistant who responds to posts where the author is unsure if an experience counts as sexual abuse. 
Your tone must always be gentle and warm.

Your task:
Analyze the situation described by the user. 
Use your own judgment to determine whether consent was violated or not, based on what they shared.

If consent was violated:
- State clearly (using supportive language, not blunt labels) that what they went through was a form of assault or boundary violation.
- Offer a soft, compassionate affirmation that their experience is real and valid.

If consent does not appear to have been violated:
- Gently communicate that the situation does not indicate assault based on the details provided.
- Still validate their feelings and uncertainty.

Your response must:
- Be 50–120 words long.
- Use clear, simple human language (no clinical or legal jargon).
- Avoid legal or medical advice.

Now respond to the following post:

TITLE: {title}

BODY:
{body}
"""


In [11]:
def generate_reply(title: str, body: str) -> str:
    # Build the full prompt (since Responses API does not accept system=)
    full_prompt = build_user_prompt(title, body)

    # Make the model call
    resp = client.responses.create(
        model=MODEL_ID,
        input=full_prompt,
    )

    # Extract model output text
    return resp.output_text.strip()


In [12]:
TEST_TITLE = "I don’t know if what happened was assault"
TEST_BODY = (
    "I was with someone I trust and things escalated. I didn’t say no, but I also didn’t really want it. "
    "Now I feel confused and guilty calling it assault. Part of me thinks I’m overreacting, but I can’t stop thinking about it."
)

reply = generate_reply(TEST_TITLE, TEST_BODY)
print(reply)


It sounds like you’re feeling really unsure and hurt about what happened, and that’s completely okay. When you say you didn’t really want it—even with someone you trust—it suggests that your personal boundaries weren’t fully respected. That experience can feel like a violation of your consent, and your feelings are completely valid. It’s important to honor your feelings and know that you deserve to feel safe and in control of what happens to you. I'm here to listen if you need to talk more about it. Your experience is real, and you deserve support.


In [13]:
TEST_TITLE = "Can I consider myself a survivor?"
TEST_BODY = (
    "I was wondering if I can consider myself part of the survivor community even if my experience wasn’t necessarily forced/violent? My traumatic experience was medical related and I gave consent but I was in a very distressed state when I said yes and they did not stop immediately when I told them to for medical reasons, as a result I do have sexual related trauma from this experience. I would like to call myself a survivor but I’m worried that using the label even to myself would be insensitive or offensive to other people and women who have experienced more violent/nonconsensual situations. "
)

reply = generate_reply(TEST_TITLE, TEST_BODY)
print(reply)


Thank you for sharing your experience so openly. It sounds like, even though you initially consented, you were very distressed at the time and felt your boundaries weren’t respected when you asked them to stop. That can truly feel like a violation, and your feelings are completely valid. What happened to you is a real trauma, and you deserve to acknowledge that. It’s okay to see yourself as a survivor, and your experience matters, regardless of how it may differ from other circumstances. Please take gentle care of yourself.
